In [1]:
import pandas as pd
from pathlib import Path
import json

In [2]:
# prefer legacy, wikipedia, official

legacy = pd.read_csv(Path("source", "attrib", "legacy_attrib.csv"))
legacy["source"] = "legacy"
legacy.head()


,person_id,photo_attribution_link,photo_attribution_text,source
0,10001,https://beta.parliament.uk/media/S3bGSTqn,© Parliament (CC-BY 3.0),legacy
1,10008,https://www.flickr.com/photos/stopclimatechaos...,https://creativecommons.org/licenses/by-nc-sa/...,legacy
2,10009,https://beta.parliament.uk/media/76klBZSN,© Parliament (CC-BY 3.0),legacy
3,10027,https://beta.parliament.uk/media/xMCoxmmw,© Parliament (CC-BY 3.0),legacy
4,10031,https://beta.parliament.uk/media/xiod6n8R,© Parliament (CC-BY 3.0),legacy


In [3]:
wiki = pd.read_csv(Path("source", "attrib", "wikidata_sources_with_attrib.csv"))
wiki = wiki[["id", "licence", "urls"]]
wiki = wiki.rename(columns={"id":"person_id", "licence":"photo_attribution_text", "urls": "photo_attribution_link"})
wiki["source"] = "wikipedia"
wiki.head()

,person_id,photo_attribution_text,photo_attribution_link,source
0,10426,Creative Commons Attribution 2.0,https://commons.wikimedia.org/wiki/File:Theres...,wikipedia
1,10633,Creative Commons Attribution-Share Alike 3.0,https://commons.wikimedia.org/wiki/File:Malcol...,wikipedia
2,10308,Public domain,https://commons.wikimedia.org/wiki/File:Glenda...,wikipedia
3,11812,Creative Commons Attribution 2.0,https://commons.wikimedia.org/wiki/File:Nichol...,wikipedia
4,24959,Creative Commons Attribution 2.0,https://commons.wikimedia.org/wiki/File:Chris_...,wikipedia


In [4]:
parl = pd.read_csv(Path("source", "attrib", "parliament_attrib.csv"))
parl["source"] = "parliament"
parl.head()

,person_id,photo_attribution_link,photo_attribution_text,source
0,10001,https://members-api.parliament.uk/api/Members/...,© Parliament (CC-BY 3.0),parliament
1,10009,https://members-api.parliament.uk/api/Members/...,© Parliament (CC-BY 3.0),parliament
2,10010,https://members-api.parliament.uk/api/Members/...,© Parliament (CC-BY 3.0),parliament
3,10011,https://members-api.parliament.uk/api/Members/...,© Parliament (CC-BY 3.0),parliament
4,10013,https://members-api.parliament.uk/api/Members/...,© Parliament (CC-BY 3.0),parliament


In [5]:
df = pd.concat([parl, wiki, legacy])
df.pivot_table("person_id", index="source", aggfunc="count")


,person_id
source,
legacy,613
parliament,1375
wikipedia,539


In [6]:
df = df.drop_duplicates("person_id")
df.pivot_table("person_id", index="source", aggfunc="count")


,person_id
source,
legacy,50
parliament,1375
wikipedia,308


In [7]:
data = (df
        .drop(columns=["source"])
        .melt("person_id", var_name="data_key", value_name="data_value"))
data.head()



,person_id,data_key,data_value
0,10001,photo_attribution_link,https://members-api.parliament.uk/api/Members/...
1,10009,photo_attribution_link,https://members-api.parliament.uk/api/Members/...
2,10010,photo_attribution_link,https://members-api.parliament.uk/api/Members/...
3,10011,photo_attribution_link,https://members-api.parliament.uk/api/Members/...
4,10013,photo_attribution_link,https://members-api.parliament.uk/api/Members/...


In [8]:
data.to_json(Path("web", "attribution.json"), orient="records")

In [14]:
df[df["photo_attribution_link"].isna()]

,person_id,photo_attribution_link,photo_attribution_text,source
209,17791,NaN,"Courtesy of Sleddall Hall Antiques, Kendal, LA...",legacy
210,18112,NaN,Peter Sandilands,legacy
213,21899,NaN,Tim Horne,legacy
244,24777,NaN,Tony Flashman,legacy
